[View in Colaboratory](https://colab.research.google.com/github/wcontractor/google_colab_notebooks/blob/master/Sentiment_Polarity.ipynb)

# Upload Data
This code was lifted from the "Hello, Colaboratory" notebook.  The files.upload method opens an explorer window to navigate directories and "Open" a file. 

In [9]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Imports

In [0]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from itertools import chain
from nltk.classify import NaiveBayesClassifier, accuracy


# Load Data
Load the data into a Pandas Dataframe.  A couple of things to note:


*   Source file has no header, so column names are created using cols and then applied with the names parameter.
*   The encoding needed to be changed to 'latin-1'.  Default utf-8 caused some parsing errror.  



In [12]:
cols = ['sentiment', 'id', 'date', 'query_string', 'user', 'text']
df = pd.read_csv('training.1600000.processed.noemoticon.csv', names=cols, encoding='latin-1')
df.head()

,sentiment,id,date,query_string,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [14]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['text_stop_words'] = df['text'].str.lower().apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['text_stop_words'][:100]

0     @switchfoot http://twitpic.com/2y1zl - awww, t...
1     upset can't update facebook texting it... migh...
2     @kenichan dived many times ball. managed save ...
3                      whole body feels itchy like fire
4     @nationwideclass no, behaving all. i'm mad. he...
5                                  @kwesidei whole crew
6                                              need hug
7     @loltrish hey long time see! yes.. rains bit ,...
8                                       @tatiana_k nope
9                                 @twittera que muera ?
10                   spring break plain city... snowing
11                                      re-pierced ears
12    @caregiving bear watch it. thought ua loss emb...
13    @octolinz16 counts, idk either. never talk any...
14    @smarrison would've first, gun. really though,...
15    @iamjazzyfizzle wish got watch you!! miss @iam...
16    hollis' death scene hurt severely watch film w...
17                                           fil

In [0]:
polarity = {'empty' : 'N',
                'sadness' : 'N',
                'enthusiasm' : 'P',
                'neutral' : 'neutral',
                'worry' : 'N',
                'surprise' : 'P',
                'love' : 'P',
                'fun' : 'P',
                'hate' : 'N',
                'happiness' : 'P',
                'boredom' : 'N',
                'relief' : 'P',
                'anger' : 'N'
         }  